# Obliczenia dla robota mobilnego
- Współrzedne
- Ograniczenia poślizgu bocznego
- [ ] Ograniczenia poślizgu wzdłużego
- Macierz Pffafa

In [25]:
from sympy import *
from IPython.display import display
# Zmienne i stale
l, l1, l2, l3 = symbols("l l_1 l_2 l_3", real=True)
a, a1, a2, a3 = symbols("a a_1 a_2 a_3", real=True)
t = symbols("t", real=True)

# Funkcje
x = Function("x")(t)
y = Function("y")(t)
xp = Function("x_p")(t)
yp = Function("y_p")(t)
yt = Function("y_t")(t)
xt = Function("x_t")(t)

phi     = Function("\\phi")(t)
theta   = Function("\\theta")(t)
xi      = Function("\\xi")(t)
dphi    = phi.diff(t).doit()
dtheta  = theta.diff(t).doit()
dxi     = xi.diff(t).doit()

# Skrocone postacie
ph, dph = phi, dphi
th, dth = theta, dtheta

In [26]:
def brak_poslizgu_bocznego(dx, dy, kat):
    global dq
    if kat == None:
        print("Brak okreslonej orientacji")
        return None
    BPB = dx * sin( kat ) - dy * cos( kat )
    print("Pierwotny wynik")
    display(BPB)
    print("Uproszczone:")
    BPB = BPB.simplify()
    display(BPB)
    print("Posegregowane:")
    BPB = collect(BPB, dq)
    display(BPB)
    return BPB

In [27]:
def macierz_pffafa(restrictions: list) -> list:
    Aq = []

    for rest in restrictions:
        A_row = []
        if rest != None:
            for dq_i in dq:
                A_row.append( rest.coeff( dq_i ) )
            Aq.append(Matrix(A_row).T)

    display(Matrix( Aq ))
    return Aq

________________________________
________________________________
## !!! Wprowadź wektor $q$


In [28]:
q = [ x, y, ph, th ]

## !!! Wprowadź (xp, yp) względem (xt, yt) lub odwrotnie:
(x,y) - oznaczają współrzedne osi względem, której wykonujemy obliczenia

In [29]:
# np.:
xt = x
yt = y
xp = x + l1*cos(ph) + l2*cos(th) 
yp = y + l1*sin(ph) + l2*sin(th)
# xs = 
# yx = 

## !!! Wprowadź orientacje osi

In [30]:
# np.:
orientacja_osi_P = th
orientacja_osi_S = None
orientacja_osi_T = ph

________________________________
________________________________

## Współrzedne (x, y)
$$ x_p, y_p, x_t, y_t $$

In [31]:
xp

l_1*cos(\phi(t)) + l_2*cos(\theta(t)) + x(t)

In [32]:
yp

l_1*sin(\phi(t)) + l_2*sin(\theta(t)) + y(t)

In [33]:
xt

x(t)

In [34]:
yt

y(t)

## Pochodne współrzędnych
$$ [ \dot{x_p}, \dot{y_p}, \dot{x_t}, \dot{y_t} ]^T $$ 

In [35]:
dxp = xp.diff(t).doit()
dxp

-l_1*sin(\phi(t))*Derivative(\phi(t), t) - l_2*sin(\theta(t))*Derivative(\theta(t), t) + Derivative(x(t), t)

In [36]:
dyp = yp.diff(t).doit()
dyp

l_1*cos(\phi(t))*Derivative(\phi(t), t) + l_2*cos(\theta(t))*Derivative(\theta(t), t) + Derivative(y(t), t)

In [37]:
dxt = xt.diff(t).doit()
dxt

Derivative(x(t), t)

In [38]:
dyt = yt.diff(t).doit()
dyt

Derivative(y(t), t)

## Wektor $ \dot{q} $

In [39]:
dq = []
for q_i in q:
    dq.append( q_i.diff(t).doit() )
Matrix(dq)

Matrix([
[     Derivative(x(t), t)],
[     Derivative(y(t), t)],
[  Derivative(\phi(t), t)],
[Derivative(\theta(t), t)]])

In [40]:
BPW_P = None; BPW_S = None; BPW_T = None
BPB_P = None; BPB_S = None; BPB_T = None

## Obliczenia ograniczeń bocznych
### BPB_P - Brak poślizgu bocznego dla osi przedniej

In [41]:
BPB_P = brak_poslizgu_bocznego(dxp, dyp, orientacja_osi_P)

Pierwotny wynik


(-l_1*sin(\phi(t))*Derivative(\phi(t), t) - l_2*sin(\theta(t))*Derivative(\theta(t), t) + Derivative(x(t), t))*sin(\theta(t)) - (l_1*cos(\phi(t))*Derivative(\phi(t), t) + l_2*cos(\theta(t))*Derivative(\theta(t), t) + Derivative(y(t), t))*cos(\theta(t))

Uproszczone:


-l_1*cos(\phi(t) - \theta(t))*Derivative(\phi(t), t) - l_2*Derivative(\theta(t), t) + sin(\theta(t))*Derivative(x(t), t) - cos(\theta(t))*Derivative(y(t), t)

Posegregowane:


-l_1*cos(\phi(t) - \theta(t))*Derivative(\phi(t), t) - l_2*Derivative(\theta(t), t) + sin(\theta(t))*Derivative(x(t), t) - cos(\theta(t))*Derivative(y(t), t)

### BPB_S - Brak poślizgu bocznego dla osi środkowej

In [42]:
# BPB_S = brak_poslizgu_bocznego(dxs, dys, orientacja_osi_S)

### BPB_T - Brak poślizgu bocznego dla osi tylniej

In [43]:
# BPB T - Brak poślizgu bocznego dla osi tylniej
BPB_T = brak_poslizgu_bocznego(dxt, dyt, orientacja_osi_T)

Pierwotny wynik


sin(\phi(t))*Derivative(x(t), t) - cos(\phi(t))*Derivative(y(t), t)

Uproszczone:


sin(\phi(t))*Derivative(x(t), t) - cos(\phi(t))*Derivative(y(t), t)

Posegregowane:


sin(\phi(t))*Derivative(x(t), t) - cos(\phi(t))*Derivative(y(t), t)

## Ograniczenia wzdłużne
### Oś przednia

In [44]:
BPW_P

### Oś środkowa

In [45]:
BPW_S

### Oś tylnia

In [46]:
BPW_T

## Macierz Pffafa

In [47]:
A1 = macierz_pffafa([ BPB_P, BPB_S, BPB_T, BPW_P, BPW_S, BPW_T ])

Matrix([
[sin(\theta(t)), -cos(\theta(t)), -l_1*cos(\phi(t) - \theta(t)), -l_2],
[  sin(\phi(t)),   -cos(\phi(t)),                             0,    0]])

In [48]:
Matrix( dq )

Matrix([
[     Derivative(x(t), t)],
[     Derivative(y(t), t)],
[  Derivative(\phi(t), t)],
[Derivative(\theta(t), t)]])